<a href="https://colab.research.google.com/github/hateley/RAG-chatbot/blob/main/rag_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building a Question and Answer system that uses RAG from drug trail information on Epkinly and Polivy. This system relies primarily on LangChain.

### Install libraries

**We need:**
* **langchain**: This is a library for GenAI. We'll use it to chain together different language models and components for our chatbot.
* **openai**: This is the official OpenAI Python client. We'll use it to interact with the OpenAI API and generate responses for our chatbot.
* **pinecone-client**: This is the official Pinecone Python client. We'll use it to interact with the Pinecone API and store our chatbot's knowledge base in a vector database.

In [8]:
# install necessary libraries
!pip install -qU \
    langchain==0.0.354 \
    openai==1.6.1 \
    pinecone-client==3.1.0 \
    tiktoken==0.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


## Make a simple chatbot first

In [9]:
#initialize the ChatOpenAI object

import os
from langchain.chat_models import ChatOpenAI
from google.colab import userdata

chat = ChatOpenAI(
    openai_api_key=userdata.get('testkey'),
    model='gpt-3.5-turbo'
)

In [10]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [12]:
#res = chat(messages)
print(res.content)

String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It posits that the fundamental building blocks of the universe are not point-like particles, but rather tiny, vibrating strings. These strings can give rise to different particles depending on their vibrational patterns.

String theory suggests that there are multiple dimensions beyond the familiar three spatial dimensions and one time dimension. The theory also proposes the existence of different vibrational modes of the strings, which correspond to different particles and forces in the universe.

One of the key ideas in string theory is the concept of supersymmetry, which posits a symmetry between particles with integer spin (bosons) and particles with half-integer spin (fermions). Supersymmetry is believed to help resolve some of the issues in particle physics, such as the hierarchy problem and unifying the fundamental forces of nature.

String theory has generate

In [13]:
# add history so we can continue the conversation

# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it has the ability to incorporate all of the fundamental forces of nature (gravity, electromagnetism, weak nuclear force, and strong nuclear force) within a single framework. In traditional particle physics, these forces are described by different theories (such as quantum field theory for the Standard Model and general relativity for gravity) that do not easily reconcile with each other.

String theory, on the other hand, offers a more comprehensive and consistent framework that can potentially describe all of these forces in a unified manner. By treating particles as vibrating strings in higher-dimensional spacetime, string theory can naturally incorporate gravity along with the other forces. This suggests that all forces and particles in the universe may emerge from a single underlying theory.

Additionally, string theory provides a way to reconcile quantum mechanics with general relativity, 

In [14]:
# right now it doesn't know about the info we want to talk about

# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What were the results of the EPCORE NHL-1 trial?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

messages.append(res)

In [15]:
print(res.content)

The EPCORE NHL-1 trial was a clinical trial evaluating the safety and efficacy of a novel treatment for patients with non-Hodgkin lymphoma (NHL). Unfortunately, as of my last update, I do not have specific information on the results of the EPCORE NHL-1 trial. Clinical trial results are typically published in scientific journals or presented at medical conferences once the study is completed.

If you are interested in the results of the EPCORE NHL-1 trial, I recommend checking clinical trial registries, medical journals, or contacting the researchers involved in the study for more information. Keep in mind that the results of clinical trials can have important implications for patient care and future research in the field of oncology.


## Import data about the clinical trials

In [20]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving epkinly_adverse-reactions.txt to epkinly_adverse-reactions.txt
Saving epkinly_clinical-trial-results.txt to epkinly_clinical-trial-results.txt
Saving epkinly_important-safety-information.txt to epkinly_important-safety-information.txt
Saving epkinly_study-design.txt to epkinly_study-design.txt
Saving polivy_important-safety-information.txt to polivy_important-safety-information.txt
Saving polivy_polarix-trial.html#trial-design.txt to polivy_polarix-trial.html#trial-design.txt
Saving polivy_summary.txt to polivy_summary.txt
Saving polivy_trial-results.txt to polivy_trial-results.txt
User uploaded file "epkinly_adverse-reactions.txt" with length 29809 bytes
User uploaded file "epkinly_clinical-trial-results.txt" with length 29133 bytes
User uploaded file "epkinly_important-safety-information.txt" with length 26388 bytes
User uploaded file "epkinly_study-design.txt" with length 30434 bytes
User uploaded file "polivy_important-safety-information.txt" with length 17077 bytes
User upl

In [16]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader()